In [1]:
from src.linear_hashing import LinearHashing
from random import sample, randint
from math import ceil


def generate_keys(amount, start=0, end=10**5):
    random_numbers = set()
    while len(random_numbers) < amount:
        random_numbers.add(randint(start, end))

    return list(random_numbers)

def average(numbers):
    return round(sum(numbers) / len(numbers), 2)


test_ps = [1, 5, 10, 20, 50]
test_alphas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


In [2]:
# Desempenho quanto ao espa¸co
for p in test_ps:
    for alpha in test_alphas:
        alphas = []
        pStars = []

        for _ in range(10):
            linear_hashing = LinearHashing(p, alpha)
            keys = generate_keys(p*1000)
            for key in keys:
                linear_hashing.insert(key)

            alphas.append(linear_hashing.alpha)
            pStars.append(linear_hashing.p_star)

        print(
            f"CONFIG: {{p: {p}, maxAlpha: {alpha}}},  avg alpha: {average(alphas)} avg p*: {average(pStars)}"
        )

CONFIG: {p: 1, maxAlpha: 0.2}  average alpha: 0.2 average p*: 1.67
CONFIG: {p: 1, maxAlpha: 0.3}  average alpha: 0.3 average p*: 1.63
CONFIG: {p: 1, maxAlpha: 0.4}  average alpha: 0.4 average p*: 1.54
CONFIG: {p: 1, maxAlpha: 0.5}  average alpha: 0.5 average p*: 1.58
CONFIG: {p: 1, maxAlpha: 0.6}  average alpha: 0.6 average p*: 1.57
CONFIG: {p: 1, maxAlpha: 0.7}  average alpha: 0.7 average p*: 1.72
CONFIG: {p: 1, maxAlpha: 0.8}  average alpha: 0.8 average p*: 1.85
CONFIG: {p: 1, maxAlpha: 0.9}  average alpha: 0.9 average p*: 2.12
CONFIG: {p: 5, maxAlpha: 0.2}  average alpha: 0.2 average p*: 1.18
CONFIG: {p: 5, maxAlpha: 0.3}  average alpha: 0.3 average p*: 1.11
CONFIG: {p: 5, maxAlpha: 0.4}  average alpha: 0.4 average p*: 1.07
CONFIG: {p: 5, maxAlpha: 0.5}  average alpha: 0.5 average p*: 1.09
CONFIG: {p: 5, maxAlpha: 0.6}  average alpha: 0.6 average p*: 1.2
CONFIG: {p: 5, maxAlpha: 0.7}  average alpha: 0.7 average p*: 1.36
CONFIG: {p: 5, maxAlpha: 0.8}  average alpha: 0.8 average p*: 1

In [3]:
# Desempenho quanto ao n´umero m´edio de acessos

for p in test_ps:
    for alpha in test_alphas:
        S_avg_accesses = []
        C_avg_accesses = []

        for _ in range(10):
            linear_hashing = LinearHashing(p, alpha)

            size = p * 1000
            sample_size = ceil(size * 0.20)

            random_keys = generate_keys(size+sample_size)

            keys = random_keys[sample_size:]
            
            for key in keys:
                linear_hashing.insert(key)

            S_sample = random_keys[:sample_size]
            C_sample = sample(keys, sample_size)

            S_sample_search_accesses = []
            C_sample_search_accesses = []
            for i in range(sample_size):
                S_sample_search_accesses.append(linear_hashing.search(S_sample[i])[1])
                C_sample_search_accesses.append(linear_hashing.search(C_sample[i])[1])

            S_avg_accesses.append(average(S_sample_search_accesses))
            C_avg_accesses.append(average(C_sample_search_accesses))
            
                
        print(
            f"CONFIG: {{p: {p}, maxAlpha: {alpha}}},  avg S accesses: {average(S_avg_accesses)} avg C accesses: {average(C_avg_accesses)}"
        )

CONFIG: {p: 1, maxAlpha: 0.2}  avg S accesses: 0.01 avg C accesses: 1.26
CONFIG: {p: 1, maxAlpha: 0.3}  avg S accesses: 0.02 avg C accesses: 1.27
CONFIG: {p: 1, maxAlpha: 0.4}  avg S accesses: 0.02 avg C accesses: 1.51
CONFIG: {p: 1, maxAlpha: 0.5}  avg S accesses: 0.27 avg C accesses: 1.51
CONFIG: {p: 1, maxAlpha: 0.6}  avg S accesses: 0.56 avg C accesses: 1.5
CONFIG: {p: 1, maxAlpha: 0.7}  avg S accesses: 0.8 avg C accesses: 1.81
CONFIG: {p: 1, maxAlpha: 0.8}  avg S accesses: 1.31 avg C accesses: 1.89
CONFIG: {p: 1, maxAlpha: 0.9}  avg S accesses: 1.8 avg C accesses: 2.07
CONFIG: {p: 5, maxAlpha: 0.2}  avg S accesses: 1.17 avg C accesses: 1.58
CONFIG: {p: 5, maxAlpha: 0.3}  avg S accesses: 1.76 avg C accesses: 1.94
CONFIG: {p: 5, maxAlpha: 0.4}  avg S accesses: 2.23 avg C accesses: 2.11
CONFIG: {p: 5, maxAlpha: 0.5}  avg S accesses: 2.9 avg C accesses: 2.44
CONFIG: {p: 5, maxAlpha: 0.6}  avg S accesses: 3.98 avg C accesses: 2.97
CONFIG: {p: 5, maxAlpha: 0.7}  avg S accesses: 4.77 avg